In [2]:
# trazendo a base que foi respondido de ura
import pymysql
import pandas as pd 
import numpy as np
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

connection = pymysql.connect(
    host='10.0.6.2',
    user='planejamento',
    password='@Planejamento!123',
    database='atn')


### Filtrando dia atual
data_inicio = '2023-11-01'
data_fim = '2023-11-03'
mes = '11'



## Trazendo todos os acordos do dia.
query = f"""
SELECT DISTINCT
    CONCAT(DDD, Fone) AS 'Telefone',
    ag.nome AS Nome_Agente,
    ag.nome_supervisor AS Nome_Supervisor,
    MotivoEncerramentoBilhete,
    ResultadoClassificacao,
    CASE
        WHEN a.grupoprincipal = 6202 THEN 'CLARO FIXO'
        WHEN a.grupoprincipal = 6204 THEN 'CLARO TV'
        WHEN a.grupoprincipal = 6335 THEN 'CLARO PJ'
        WHEN a.grupoprincipal = 6203 THEN 'CLARO MOVEL'
        WHEN a.grupoprincipal = 6200 THEN 'NET PRED'
        WHEN a.grupoprincipal = 6336 THEN 'NET PJ'
	    WHEN a.grupoprincipal = 6201 THEN 'GEVENUE PRED'
        WHEN a.grupoprincipal = 6337 THEN 'GEVENUE PJ'
            END AS Filas_URA,
    DATE_FORMAT(a.instante, '%d-%m-%y') AS DATA
FROM
    totalinfo_2023_{mes} a
JOIN
    agentes ag ON a.agentederivacao = ag.username
WHERE
    a.instante >= '{data_inicio} 08:00:00'
    AND   a.instante <= '{data_fim} 23:00:00'
	AND a.ResultadoClassificacao = 'vozhumana'
	AND a.OrigemChamada in ('discador','entrante') 
    AND a.Agente <> ''
    AND a.GrupoFila IN (6202,6204,6335,6203,6200,6336,6201,6337)
	AND ag.nome NOT LIKE 'AV%'
    AND MotivoEncerramentoBilhete <> 'fimapl'
    
	union
      	
    SELECT DISTINCT
    CONCAT(DDD, Fone) AS 'Telefone',
    ag.nome AS Nome_Agente,
    ag.nome_supervisor AS Nome_Supervisor,
    MotivoEncerramentoBilhete,
    a.ResultadoClassificacao,
    CASE
        WHEN a.GrupoFila = 6342 THEN 'CLARO RECEPTIVO ATN'
        WHEN a.GrupoFila = 6196 THEN 'CLARO RECEPTIVO ATN'
        WHEN a.GrupoFila = 6198 THEN 'NET RECEPTIVO MUTANT'
        WHEN a.GrupoFila = 6194 THEN 'GEVENUE  REC'
            END AS Filas_URA,
    DATE_FORMAT(a.instante, '%d-%m-%y') AS DATA
FROM
    totalinfo_2023_{mes} a
JOIN
    agentes ag ON a.agentederivacao = ag.username
WHERE
    a.instante >= '{data_inicio} 08:00:00'
    AND   a.instante <= '{data_fim} 23:00:00'
	AND a.tempoconversacao_ms > 0
    AND a.Agente <> ''
    AND a.GrupoFila IN (6342,6196,6198,6194)
	AND ag.nome NOT LIKE 'AV%'
    AND MotivoEncerramentoBilhete <> 'fimapl'
    AND a.ResultadoClassificacao <> 'vozhumana'
"""



chamadas_teles_claro = pd.read_sql(query, connection)
# chamadas_teles_claro = chamadas_teles_claro.drop_duplicates(['CODIGO UNICO CHAMADA'])

connection.close()

In [ ]:
chamadas_teles_claro.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Dezembro\5\Markin 7\respondido/chamadas_teles_0512")